In [4]:
import os
import json
import re
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from pinecone import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableMap, RunnablePassthrough
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy

# 1) 환경 변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")
INDEX_NAME = "card-index"

# 2) 카드 데이터 로드
with open("cards.json", encoding="utf-8") as f:
    cards = json.load(f)
card_lookup = {str(card["id"]): card for card in cards}

# 3) Pinecone 초기화 및 임베딩 준비
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)
pinecone_index = pc.Index(INDEX_NAME)
embedder = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

# 4) LLM, 파서 준비
llm = ChatOpenAI(model_name="gpt-4.1", openai_api_key=OPENAI_API_KEY)
parser = StrOutputParser()

# --- 쿼리 확장 함수 --- #
def expand_queries(base_query: str) -> list[str]:
    prompt = (
        f"질문: {base_query}\n"
        "위 질문과 '의미는 같지만' 표현 방식만 다른 질문을 4개 생성해줘. "
        "각 질문은 원래 질문의 조건(브랜드, 연회비, 혜택 등)을 모두 반드시 포함해야 해. "
        "카드 혜택 추천에 사용할 것이고, 너무 두루뭉술하게 말하지 말고, 각각 다른 단어, 어순, 표현법을 써줘. "
        "답변은 아래와 같이 번호 없이 한 줄씩 출력해줘.\n"
        "- 예: 배달음식 할인 카드 추천해줘\n"
        "- 예: 배달앱 할인 신용카드 중 좋은 거 있어?\n"
    )
    response = llm.invoke(prompt)
    queries = [base_query] + [line.strip("- ") for line in response.content.strip().splitlines() if line.strip()]
    return queries[:5]

# --- LLM에서 메타데이터 필터 추출 (프롬프트 강화) --- #
def get_filter_json_via_llm(query: str) -> dict:
    prompt = f"""
아래 "질문"에서 **명시적으로 드러난** 카드 조건만 JSON으로 추출해줘.
카드명/카드사/브랜드/연회비 같은 특정 값은 **질문에 직접 등장할 때만** 채워줘.
질문에 등장하지 않은 항목은 null로 남겨.
특정 카드명/브랜드/연회비는 **추정, 예시 추가 절대 금지**.
리스트/배열 형태도 금지. 반드시 하나의 딕셔너리(JSON)만 출력.
예시:
질문: 배달 혜택 많은 카드 추천해줘
→
{{
  "name": null,
  "brand": null,
  "global_brand": null,
  "fee_domestic": null,
  "fee_global": null
}}
질문: 연회비 5만원 이하의 삼성카드 추천해줘
→
{{
  "name": null,
  "brand": "삼성카드",
  "global_brand": null,
  "fee_domestic": {{"op": "lte", "value": 50000}},
  "fee_global": {{"op": "lte", "value": 50000}}
}}
질문: 현대카드 중 연회비 2만원 이하, Visa 브랜드 추천해줘
→
{{
  "name": null,
  "brand": "현대카드",
  "global_brand": "Visa",
  "fee_domestic": {{"op": "lte", "value": 20000}},
  "fee_global": {{"op": "lte", "value": 20000}}
}}

질문: {query}
반드시 위 예시 포맷을 따르고, JSON 외 다른 출력 절대 금지.
만약 둘중하나라도 만족하면 출력가능하게끔.
"""
    response = llm.invoke(prompt)
    raw = response.content.strip()

    # 백틱 제거
    if raw.startswith("```json") or raw.startswith("```"):
        raw = re.sub(r"```(?:json)?", "", raw).strip()
        raw = raw.rstrip("`").strip()
    try:
        result = json.loads(raw)
        if isinstance(result, list) and len(result) > 0 and isinstance(result[0], dict):
            result = result[0]
        return result
    except Exception as e:
        print("LLM JSON 파싱 실패:", e)
        print("응답 내용:", response.content)
        return {}

# --- 메타데이터 필터 변환 --- #
def build_metadata_filter(parsed) -> dict:
    if isinstance(parsed, list):
        if len(parsed) > 0 and isinstance(parsed[0], dict):
            parsed = parsed[0]
        else:
            return {}

    filter_dict = {}
    or_conditions = []

    if parsed.get("name"):
        filter_dict["name"] = {"$eq": parsed["name"]}
    if parsed.get("brand"):
        filter_dict["brand"] = {"$eq": parsed["brand"]}
    if parsed.get("global_brand"):
        filter_dict["global_brand"] = {"$eq": parsed["global_brand"]}

    # 연회비 조건을 OR로 묶기
    for fee_field, pinecone_field in [("fee_domestic", "annual_fee_domestic"), ("fee_global", "annual_fee_global")]:
        fee_obj = parsed.get(fee_field)
        if isinstance(fee_obj, dict):
            op = fee_obj.get("op")
            val = fee_obj.get("value")
            if op in ["lte", "gte", "eq"] and isinstance(val, (int, float)):
                or_conditions.append({pinecone_field: {f"${op}": val}})

    # 둘 다 조건 있으면 OR로 묶음
    if or_conditions:
        if filter_dict:
            # AND(기타조건) & OR(연회비)
            filter_dict["$or"] = or_conditions
            return filter_dict
        else:
            # 연회비만 있을 때
            return {"$or": or_conditions}
    return filter_dict


# --- 쿼리 임베딩 --- #
def embed_multiple_queries(queries):
    vectors = embedder.embed_documents(queries)
    avg_vector = [sum(col) / len(col) for col in zip(*vectors)]
    return avg_vector

expand_and_embed = RunnableLambda(lambda q: embed_multiple_queries(expand_queries(q)))

# --- 유사도 검색 + 필터 적용 --- #
def search_similar_cards_with_filter(input: dict, k=5):
    query = input["query"]
    vector = input["vector"]
    parsed = get_filter_json_via_llm(query)
    metadata_filter = build_metadata_filter(parsed)
    if metadata_filter:
        resp = pinecone_index.query(vector=vector, top_k=k, include_metadata=True, filter=metadata_filter)
    else:
        resp = pinecone_index.query(vector=vector, top_k=k, include_metadata=True)
    return [card_lookup[match["id"]] for match in resp["matches"] if match["id"] in card_lookup]

# --- 카드 설명 포맷 --- #
def format_cards(cards):
    info_strs = []
    for card in cards:
        name = card.get("name", "")
        issuer = card.get("brand", "")
        card_id = card.get("id", "")
        annual_domestic = f"{card.get('annual_fee_domestic', 0):,}"
        annual_global = f"{card.get('annual_fee_global', 0):,}"
        summary_box = card.get("summary_box", "")
        benefits = "\n".join(f"- {b['category']}: {' / '.join(b['details'])}" for b in card.get("benefits", []))
        info_strs.append(
            f"카드명: {name}\n"
            f"카드ID: {card_id}\n"
            f"카드사: {issuer}\n"
            f"연회비(국내): {annual_domestic}원\n"
            f"연회비(해외): {annual_global}원\n"
            f"요약: {summary_box}\n"
            f"혜택:\n{benefits}"
        )
    return "\n\n".join(info_strs)

format_card_text = RunnableLambda(format_cards)

# --- 프롬프트 생성 --- #
def make_prompt(input: dict) -> str:
    return (
        f"질문: {input['query']}\n\n"
        "아래 카드 정보(context)에 명시된 내용만 참고하여 답변해 주세요.\n"
        "질문에 해당하는 카드가 여러 개여도 아래 context에 나온 카드 중 제일 질문에 적합한 카드로 한개 추천하세요.\n"
        "context에 나와 있지 않은 정보(연회비, 카드사, 혜택 등)는 절대로 답변에 포함하지 마세요.\n"
        "카드 정보 외 추가 설명, 일반적인 안내, 배경지식, 상상, 유추 등도 하지 마세요.\n"
        "답변은 아래 context에 나와 있는 카드 정보 중 하나의 전체 정보를 가시적으로 보기 좋게 보여주세요.\n"
        "내용 자체는 너무 길지않게 적당히 요약해서 보여주세요\n"
        "[카드명:00카드\n 카드사: 00카드 \n연회비(국내): 1234원\n연회비(해외):1234원\n글로벌브랜드: visa\nurl:https://www.card-gorilla.com/card/detail/카드id\n혜택:\n"
        "요약은 하되 위 규격에 맞춰서 출력해주세요.\n"
        "주어진 데이터에 질문에 맞는 카드정보가 없으면 **정보가 부족합니다** 라고 입력해주세요"
        f"{input['cards_block']}"
    )


# --- 체인 구성 --- #
recommend_chain = (
    RunnablePassthrough()
    | {"query": RunnablePassthrough(), "vector": expand_and_embed}
    | RunnableMap({
        "query": lambda x: x["query"],
        "cards": search_similar_cards_with_filter
    })
    | RunnableMap({
        "query": lambda x: x["query"],
        "cards_block": lambda x: format_cards(x["cards"])
    })
    | RunnableLambda(make_prompt)
    | llm
    | parser
)


# --- robust 카드ID 추출: 카드ID → fallback 카드명 --- #
def get_card_ids_from_answer(answer):
    """
    추천 결과에서 카드ID(있으면) 추출, 없으면 카드명 기준으로 cards.json에서 찾기
    """
    card_ids = []
    card_names = []
    for line in answer.splitlines():
        line_clean = line.strip(" -\t")
        # 카드ID 직접 파싱
        if line_clean.startswith("카드ID:"):
            cid = line_clean.replace("카드ID:", "").strip()
            if cid.isdigit():
                card_ids.append(cid)
        elif line_clean.startswith("카드명:"):
            name = line_clean.replace("카드명:", "").strip()
            card_names.append(name)
    # 카드ID 우선, 없으면 카드명 → card_lookup에서 매칭
    if not card_ids and card_names:
        for name in card_names:
            for card in cards:
                if card["name"] == name:
                    card_ids.append(str(card["id"]))
                    break
    return card_ids

# --- MAIN LOOP --- #
def run_single_ragas_eval(query, answer, top_k=5):
    """단일 쿼리에 대한 ragas 평가"""
    vector = embed_multiple_queries([query])
    resp = pinecone_index.query(vector=vector, top_k=top_k, include_metadata=True)
    doc_texts = []
    for match in resp["matches"]:
        card_id = match["id"]
        card = card_lookup.get(card_id)
        if card:
            doc_texts.append(format_cards([card]))
    dataset = Dataset.from_dict({
        "question": [query],
        "answer": [answer],
        "contexts": [doc_texts],
    })
    result = evaluate(
        dataset=dataset,
        metrics=[answer_relevancy]
    )
    return result.to_pandas()

def main():
    print("=== 카드 추천 챗봇 ===\n")
    while True:
        user_query = input("\n💬 추천 받고 싶은 카드를 설명해 주세요 (종료: q): ").strip()
        if user_query.lower() in ("q", "quit", "exit"):
            print("\n프로그램을 종료합니다.")
            break
        # 추천 결과 호출 (예외처리)
        try:
            answer = recommend_chain.invoke(user_query)
        except Exception as e:
            print(f"[오류] 추천 결과 생성에 실패했습니다: {e}")
            continue

        print(f"질문 : {user_query}\n\n[카드 추천 결과]\n{answer}\n{'-'*40}")

        # === 정확도(RAGAS) 평가결과 자동 출력 ===
        try:
            eval_df = run_single_ragas_eval(user_query, answer)
            print("[정확도 평가 결과 (RAGAS)]", flush=True)
            print(eval_df[["answer_relevancy"]], flush=True)
        except Exception as e:
            print(f"[RAGAS 평가 오류] {e}")
        print("-" * 40)

        # 메뉴 반복
        while True:
            print("\n원하는 동작을 선택하세요:", flush=True)
            print("1. 재검색(새로운 쿼리)", flush=True)
            print("2. 종료", flush=True)
            menu = input("메뉴 번호 입력: ").strip()
            if menu == "1":
                break  # while True -> 재검색
            elif menu == "2":
                print("\n프로그램을 종료합니다.")
                return
            else:
                print("잘못된 입력입니다. 다시 선택해 주세요.")

if __name__ == "__main__":
    main()

=== 카드 추천 챗봇 ===

질문 : 연회비 5만원 이상의 삼성카드중 비자카드 추천해줘

[카드 추천 결과]
카드명: THE O V2 (포인트)  
카드사: 삼성카드  
연회비(국내): 595,000원  
연회비(해외): 600,000원  
글로벌브랜드: visa  
url: https://www.card-gorilla.com/card/detail/1904  
혜택:  
- 연 1회 11가지 프리미엄 기프트(항공권, 호텔, 골프, 쇼핑 등) 중 택1 (최대 35만원 상당)  
- 모든 가맹점 1.2% 빅포인트 적립 (한도 없이 제공)  
- 10대 커피전문점 20% 청구할인(월 1만원 한도, 전월 50만원 이상 이용 시)  
- 영화관 5,000원 청구할인(월 1회, 전월 50만원 이상 이용 시)  
- 택시 3,000원 청구할인(월 1회, 전월 50만원 이상 이용 시)  
- 국내외 공항 라운지 무료, 공항 및 호텔 발렛파킹 무료, Visa Infinite 프리미엄 서비스  
- 전월 실적 조건: 일부 서비스는 전월 일시불/할부 50만원 이상 이용 시 제공
----------------------------------------


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

[정확도 평가 결과 (RAGAS)]
   answer_relevancy
0          0.756955
----------------------------------------

원하는 동작을 선택하세요:
1. 재검색(새로운 쿼리)
2. 종료
잘못된 입력입니다. 다시 선택해 주세요.

원하는 동작을 선택하세요:
1. 재검색(새로운 쿼리)
2. 종료
질문 : 연회비가 5만원 이상, 10만원 이하면서, VISA 브랜드이면서 삼성카드인 것을 추천해줘

[카드 추천 결과]
**정보가 부족합니다**
----------------------------------------


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

[정확도 평가 결과 (RAGAS)]
   answer_relevancy
0               0.0
----------------------------------------

원하는 동작을 선택하세요:
1. 재검색(새로운 쿼리)
2. 종료

프로그램을 종료합니다.
